In [9]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import (
                                                        AzureSearch,
                                                        AzureSearchVectorStoreRetriever
                                                        )
from ragas.llms import LangchainLLM
from ragas import evaluate
from ragas.metrics.critique import harmfulness
from datasets import load_dataset

In [2]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["azure_endpoint"] = "https://ai-dev-openai-fts.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "7439c61b0745446dbbb029521f498265"

In [14]:
azure_model = AzureChatOpenAI(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_version = "2023-07-01-preview",
    openai_organization = "azure",
    temperature=0,    
    deployment_name="gpt35")

In [15]:
azure_embeddings = AzureOpenAIEmbeddings(
            openai_api_key = "7439c61b0745446dbbb029521f498265",
            azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
            openai_api_type = "azure",
            openai_api_version = "2023-07-01-preview",
            deployment="text-embedding-ada-002", 
            chunk_size=1
                                )

In [16]:
query = "What did the president say about Justice Breyer"

azure_model.predict(query)

"As an AI language model, I do not have access to the latest news and updates. However, as of my last training data, I do not have any information about the president's recent comments on Justice Breyer."

In [6]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)
    

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

documents

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [10]:
acs = AzureSearch(azure_search_endpoint="https://ai-dev-ss-fts.search.windows.net",
                 azure_search_key="ICyDmM0S3sWaLpYwNGPfUOz7jblRyOHnE6p7uDHPRRAzSeAaLBFM",
                 index_name="reg-data-ankur",
                 embedding_function= azure_embeddings.embed_query
                 )

In [11]:
acs.add_documents(documents=chunks)

['NDcyOTIzOGItMDAxZi00NTdlLWJkODgtNmY2YmZhZGFjYTgz',
 'MTEwNGRhOTItOTJiYS00MzZhLWJiY2YtZThmMDdhMTViNzU5',
 'OTQ1NTM1NDEtYjRhNi00ZTA1LTlmMDQtMTZhZDAzZGFiM2Vj',
 'NGFkMjZlNjYtMDdiNS00YjYyLWJmMGUtNDMzNmJhMWIyNzRk',
 'MmI1MjZlYjktYzlkYS00YTQzLTkwYTgtNjU4OWFkYTM3Yjc4',
 'MTRiOWIzMzQtNzYyZS00YzkwLTkzODItNjI4ZDYyZmYwNDdm',
 'YjJkMmUzMjQtZjQ5My00NTFmLTgzODMtMGYzNTA4ZDZlMTU5',
 'M2EyN2RkNjEtNGVjNC00NTdhLTg5MTAtZjI4MjJmMjIzZTYw',
 'MTIzN2ZiYmEtODQ3YS00NzQxLTg0YWEtY2YxNWRiMWE4NmI4',
 'MDMyZGY4ZGQtN2U4MS00NTdhLWE0YWEtNzI2OTI0M2MwOWQy',
 'ODMyM2RjOGEtMThmOC00MTY2LWEzMWUtMTI5MWE3OWJjNzli',
 'ZWRkNjA2NTYtMWIxYy00NDRhLTk0OWItODdhNjlkYjRkOGJl',
 'MTNmYTM3ZDQtOTBjMi00NjFiLTg0MzItNzJhODU3Y2NmNDM2',
 'NTMwMDVjOTAtNWM4NS00YWJhLWFkMTUtOTgyYjhkM2Q1NjMw',
 'Y2U1ZjJjNTgtMzg4Yi00Y2ZkLTk1NGYtNjg4ZDYyZjE2MmYw',
 'YjI2OWFjMmItMjljNi00YmQ0LWE0ZDEtMmQwMzE4YjAyY2Ez',
 'ZjM4YzVlOWEtMWUxZi00ZjYwLWFkOTQtODdhNDkyMGQ1ZWQ0',
 'ZDE1ZjhkNTYtNTA4NC00ZmM5LWFkNmEtYjgxZTFmMTllZjEz',
 'NjA4NjRmN2YtZDAxYy00OTg2LWIzODctYmIzMTFiNDEz

In [12]:
retriever = acs.as_retriever(search_kwargs={"k": 3})

In [13]:
from langchain import PromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"


In [23]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | azure_model
    | StrOutputParser() 
)

In [24]:
# Ragas wants ['question', 'answer', 'contexts', 'ground_truths'] as
'''
{
    "question": [], <-- question from faq doc
    "answer": [], <-- answer from generated result
    "contexts": [], <-- context
    "ground_truths": [] <-- actual answer
}
'''

from datasets import Dataset

questions = ["What did the president say about Justice Breyer?", 
             "What did the president say about Intel's CEO?",
             "What did the president say about gun violence?",
            ]
ground_truths = [["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
                ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
                ["The president asked Congress to pass proven measures to reduce gun violence."]]
answers = []
contexts = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [25]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 3
})

In [17]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

In [19]:
# wrapper around azure_model
ragas_azure_model = LangchainLLM(azure_model)
# patch the new RagasLLM instance
answer_relevancy.llm = ragas_azure_model

In [20]:
# embeddings can be used as it is
answer_relevancy.embeddings = azure_embeddings

In [21]:
for m in metrics:
    m.__setattr__("llm", ragas_azure_model)

In [26]:
result = evaluate(
    dataset,
    metrics=metrics,
)

result

evaluating with [faithfulness]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.47s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:06<00:00,  6.96s/it]


...

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


evaluating with [harmfulness]


100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


{'faithfulness': 0.6667, 'answer_relevancy': 0.2593, 'context_recall': 1.0000, 'context_precision': 0.3333, 'harmfulness': 0.0000}

In [27]:
df = result.to_pandas()
df.head()

question  \
0  What did the president say about Justice Breyer?   
1     What did the president say about Intel's CEO?   
2    What did the president say about gun violence?   

                                              answer  \
0  The president thanked Justice Stephen Breyer f...   
1  The president did not say anything specific ab...   
2  The president called on Congress to pass measu...   

                                            contexts  \
0  [Tonight, I’d like to honor someone who has de...   
1  [But that’s just the beginning. \n\nIntel’s CE...   
2  [And I ask Congress to pass proven measures to...   

                                       ground_truths  faithfulness  \
0  [The president said that Justice Breyer has de...           1.0   
1  [The president said that Pat Gelsinger is read...           0.0   
2  [The president asked Congress to pass proven m...           1.0   

   answer_relevancy  context_recall  context_precision  harmfulness  
0          0.000000             1.0                0.0            0  
1          0.000000             1.0                0.0            0  
2          0.777788             1.0                1.0            0